In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<h1> Extracting Landmark </h1>

In [7]:
def calculateAngle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radian = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radian*180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

<h1> Mediapipe for Video </h1>

RUN THIS ONLY FOR DATA COLLECTION

In [13]:
cap = cv2.VideoCapture("../PoseVideos/LCW_Cropped-5.mp4")
crop_rate = 3
under_arm_degree = 30
# Mediapipe Instance
with mp_pose.Pose(static_image_mode = False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    last_pose = "Unknown"
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        # Recoloring Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detection
        results = pose.process(image)

        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract

        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            left_elbow =  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Angles
            left_arm = calculateAngle(left_shoulder, left_elbow, left_wrist)
            right_arm = calculateAngle(right_shoulder, right_elbow, right_wrist)

            left_leg = calculateAngle(left_hip, left_knee, left_ankle)
            right_leg = calculateAngle(right_hip, right_knee, right_ankle)

            left_under_arm = calculateAngle(left_elbow, left_shoulder, left_hip)
            right_under_arm = calculateAngle(right_elbow, right_shoulder, right_hip)

            if (165 <= right_under_arm <= 180):
               last_pose = "Smash"

            else:
                cv2.putText(image, last_pose, [20, 20], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            x, y, z = image.shape

            if right_under_arm > under_arm_degree and count % crop_rate == 0:
                try:
                    os.mkdir("created")
                except:
                    pass
                img_path = "created"

                
                img_name = 'cropped_' + str(count) + '.png'
                img_path = os.path.join(img_path, img_name )
                cv2.imwrite(img_path, image)
            count += 1
        except:
            pass    

        # Render Detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

<h1> MediaPipe for Image </h1>

THIS SECTION IS FOR COLLECTING DATA FROM IMAGES THAT HAS BEEN CLASSIFIED

In [1]:
folderDirs = ['backhand', 'forehand', 'netdrop', 'serve']

In [4]:
crop_rate = 15
path = "created/"
filler = "data"
# Mediapipe Instance
# left_shoulder, right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle,
f = open("data.csv", "w")
f.writelines("label, left_arm, right_arm, left_leg, right_leg, left_under_arm, right_under_arm\n")
f.close()
t = cv2.imread("backhand/cropped_1035.png");
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    last_pose = "Unknown"
    count = 0
    for i in range(len(folderDirs)):
        f = open("data.csv", "a")
        
        for eaImg in os.listdir(filler + "/" +folderDirs[i] + "/"):
            print(filler + "/" + folderDirs[i] + "/" + eaImg)
            frame = cv2.imread(filler + "/" +folderDirs[i] + "/" + eaImg)
            # ret, frame = cap.read()

            # Recoloring Image
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detection
            results = pose.process(image)

            # Recolor back
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract
            try:
                landmarks = results.pose_landmarks.landmark
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                left_elbow =  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                # Angles
                left_arm = calculateAngle(left_shoulder, left_elbow, left_wrist)
                right_arm = calculateAngle(right_shoulder, right_elbow, right_wrist)

                left_leg = calculateAngle(left_hip, left_knee, left_ankle)
                right_leg = calculateAngle(right_hip, right_knee, right_ankle)

                left_under_arm = calculateAngle(left_elbow, left_shoulder, left_hip)
                right_under_arm = calculateAngle(right_elbow, right_shoulder, right_hip)

                f.writelines(str(folderDirs[i]) + ", " +
                             str(left_arm) + ", " +
                             str(right_arm) + ", " +
                             str(left_leg) + ", " +
                             str(right_leg) + ", " +
                             str(left_under_arm) + ", " +
                             str(right_under_arm) + "\n")
            except:
                pass
            # Render Detection
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # cv2.imshow('MediaPipe', image)
            try:
                os.mkdir("estimated")
            except:
                pass
            img_path = "estimated"
            img_name = 'pose_' + str(count) + '.png'
            img_path = os.path.join(img_path, img_name )
            cv2.imwrite(img_path, image)
            count += 1

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

f.close()

# cap.release()
cv2.destroyAllWindows()

data/backhand/cropped_1035.png
data/backhand/cropped_1040.png
data/backhand/cropped_1071.png
data/backhand/cropped_1074.png
data/backhand/cropped_1224.png
data/backhand/cropped_1227.png
data/backhand/cropped_1230.png
data/backhand/cropped_1233.png
data/backhand/cropped_1287.png
data/backhand/cropped_1290.png
data/backhand/cropped_1293.png
data/backhand/cropped_1296.png
data/backhand/cropped_1299.png
data/backhand/cropped_1377.png
data/backhand/cropped_1380.png
data/backhand/cropped_189.png
data/backhand/cropped_190.png
data/backhand/cropped_192.png
data/backhand/cropped_195.png
data/backhand/cropped_2030.png
data/backhand/cropped_2035.png
data/backhand/cropped_204.png
data/backhand/cropped_207.png
data/backhand/cropped_210.png
data/backhand/cropped_243.png
data/backhand/cropped_246.png
data/backhand/cropped_249.png
data/backhand/cropped_2613.png
data/backhand/cropped_2616.png
data/backhand/cropped_2619.png
data/backhand/cropped_2622.png
data/backhand/cropped_3081.png
data/backhand/crop

<h1> Predicting with Model </h1>
<p> v01 </p>

In [30]:
import pickle
import pandas as pd

<h3>MODEL TO CHOOSE:<h3>
<ul>
<li style="font-size:16px">badminton_modelv01.pkl : Initial Model that has 70% accuracy with less data</li>
<li style="font-size:16px">badminton_modelv02.pkl : 6 classifications: Forehand, Backhand, Serve, Smash, Netdrop, 67% accuracy using Random Forest Classification</li>
<li style="font-size:16px">badminton_modelv03.pkl : 3 classifications: Forehand, Backhand, Netdrop. 62% accuracy using Random Forest Classification.</li>
<br>
<li style="font-size:16px">fillerv01.pkl: 5 Classifications, 60% - 64% max at 64.5% with RFC</li>
<li style="font-size:16px">fillervo2.pkl: 4 Classifications, constant of 67% with LR </li>
<li style="font-size:16px">fillerv03.pkl: 3 Classifications, RFC of 62% - 70% average of 65% using RFC</li>
<ul>

In [33]:
with open('badminton_modelv03.pkl', 'rb') as f:
    model = pickle.load(f)

In [35]:
cap = cv2.VideoCapture("../PoseVideos/badminton2.mp4")
crop_rate = 5
# Mediapipe Instance
with mp_pose.Pose(static_image_mode = False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    last_pose = "Unknown"
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()

        # Recoloring Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detection
        results = pose.process(image)

        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract

        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            left_elbow =  [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]

            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Angles
            left_arm = calculateAngle(left_shoulder, left_elbow, left_wrist)
            right_arm = calculateAngle(right_shoulder, right_elbow, right_wrist)

            left_leg = calculateAngle(left_hip, left_knee, left_ankle)
            right_leg = calculateAngle(right_hip, right_knee, right_ankle)

            left_under_arm = calculateAngle(left_elbow, left_shoulder, left_hip)
            right_under_arm = calculateAngle(right_elbow, right_shoulder, right_hip)
            # print(left_arm, right_arm, left_leg, right_leg, left_under_arm, right_under_arm)
            lst = np.array([left_arm, right_arm, left_leg, right_leg, left_under_arm, right_under_arm]).flatten()
            lst = list(lst)
            x = pd.DataFrame([lst])
            # if len(x.values[0]) < 6:
            #     print(x)
            # x = pd.DataFrame([left_arm, right_arm, left_leg, right_leg, left_under_arm, right_under_arm])
            model_class = model.predict(x.values)[0]
            model_prob = model.predict_proba(x.values)[0]
            # # print(x)
            
            if(round(model_prob[np.argmax(model_prob)],2) > 0.7):

                # Display Class
                cv2.putText(image, model_class, [25, 25], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Unknown", [25, 25], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            # Display Probability
            cv2.putText(image, str(round(model_prob[np.argmax(model_prob)],2)), [35, 45], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass    

        # Render Detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()